In [ ]:
# Load required libraries
library(tximport)
library(DESeq2)
library(tidyverse) 
library(rtracklayer)
library(rstatix)



In [ ]:
# Import GTF
gtf <- import("genome/gencode.v47.basic.annotation.gtf.gz")

# Create tx2gene mapping WITHOUT stripping version numbers
tx2gene <- data.frame(
    TXNAME = gtf$transcript_id[gtf$type == "transcript"],  # Keep versions
    GENEID = gtf$gene_id[gtf$type == "transcript"],       # Keep versions
    SYMBOL = gtf$gene_name[gtf$type == "transcript"],
    TX_NAME = gtf$transcript_name[gtf$type == "transcript"],
    GENE_TYPE = gtf$gene_type[gtf$type == "transcript"]
)

# Verify the format matches
head(tx2gene)  # Should now show IDs with version numbers

In [ ]:
sample_info <- read.csv('sample_info.csv', row.names=1)

sample_info

In [ ]:
# Setup file paths for the new structure
files <- file.path("salmon_quant", rownames(sample_info), "quant.sf")
names(files) <- rownames(sample_info)

# Verify files exist
all(file.exists(files))

tx2gene_protein_coding <- tx2gene[tx2gene$GENE_TYPE == "protein_coding",]


# Import with tximport
txi <- tximport(files, 
                type = "salmon", 
                tx2gene = tx2gene_protein_coding[,c("TXNAME", "GENEID")],
                ignoreTxVersion = FALSE,
                txOut = FALSE)


In [ ]:
str(txi)

In [ ]:
library(ggplot2)
library(patchwork)

# Calculate millions of counts and TPMs per sample
millions_counts_df <- data.frame(
    sample = colnames(txi$counts),
    millions = colSums(txi$counts)/1e6
) %>%
  mutate(
    condition = sample_info[sample, "condition"],
    cell_type = sample_info[sample, "cell_type"]
  )

millions_tpm_df <- data.frame(
    sample = colnames(txi$abundance),
    millions = colSums(txi$abundance)/1e6
) %>%
  mutate(
    condition = sample_info[sample, "condition"],
    cell_type = sample_info[sample, "cell_type"]
  )

# Create counts barplot
p1 <- ggplot(millions_counts_df, aes(x = sample, y = millions, fill = condition)) +
  geom_bar(stat = "identity") +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(
    title = "Millions of Raw Counts per Sample",
    x = "Sample",
    y = "Millions of Reads"
  )

# Create TPM barplot
p2 <- ggplot(millions_tpm_df, aes(x = sample, y = millions, fill = condition)) +
  geom_bar(stat = "identity") +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(
    title = "Sum of TPM Values per Sample",
    x = "Sample",
    y = "Millions of TPM"
  )

# Display plots one above the other
p1 / p2

In [ ]:
# First reshape data for rstatix
tidy_tpm <- as_tibble(tpm, rownames = "gene") %>%
  pivot_longer(-gene, 
               names_to = "sample", 
               values_to = "tpm") %>%
  mutate(condition = ifelse(grepl("Mock", sample), "Mock", "SARS-CoV-2"))

# Run t-tests for all genes
t_test_results <- tidy_tpm %>%
  group_by(gene) %>%
  t_test(tpm ~ condition) %>%
  select(gene, statistic, p) %>%
  rename(pvalue = p)

# Create final test_data tibble
test_data <- tidy_tpm %>%
  group_by(gene) %>%
  summarise(
    mean_mock = mean(tpm[condition == "Mock"]),
    mean_sars = mean(tpm[condition == "SARS-CoV-2"]),
    var_mock = var(tpm[condition == "Mock"]),
    var_sars = var(tpm[condition == "SARS-CoV-2"])
  ) %>%
  # Add gene symbols
  left_join(
    tibble(
      gene = tx2gene$GENEID,
      symbol = tx2gene$SYMBOL
    ),
    by = "gene"
  ) %>%
  # Add t-test results
  left_join(t_test_results, by = "gene") %>%
  # Calculate MA plot values and other metrics
  mutate(
    M = log2(mean_sars/mean_mock),
    A = log2((mean_sars + mean_mock)/2),
    log2FC = M,
    mean_expr = (mean_mock + mean_sars)/2,
    padj = p.adjust(pvalue, method = "BH")
  ) #%>%
# Finally pivot back to wide format
 # pivot_wider(
 #   id_cols = c(gene, symbol, M, A, log2FC, mean_expr, statistic, pvalue, padj),  # keep these columns as is
 #   names_from = sample,  # pivot on sample names
  #  values_from = tpm     # the TPM values
  ##)

In [ ]:
test_data

In [ ]:

# First reshape data for rstatix
tidy_tpm <- data.frame(tpm) %>%
  rownames_to_column("gene") %>%
  pivot_longer(-gene, 
               names_to = "sample", 
               values_to = "tpm") %>%
  mutate(condition = ifelse(grepl("Mock", sample), "Mock", "SARS-CoV-2"))

# Run t-tests for all genes
t_test_results <- tidy_tpm %>%
  group_by(gene) %>%
  t_test(tpm ~ condition) %>%
  select(gene, statistic, p) %>%
  rename(pvalue = p)

# Start with already tidy tpm data from before
test_data <- tidy_tpm %>%
  group_by(gene) %>%
  summarise(
    mean_mock = mean(tpm[condition == "Mock"]),
    mean_sars = mean(tpm[condition == "SARS-CoV-2"]),
    var_mock = var(tpm[condition == "Mock"]),
    var_sars = var(tpm[condition == "SARS-CoV-2"])
  ) %>%
  # Add gene symbols
  left_join(
    data.frame(
      gene = tx2gene$GENEID,
      symbol = tx2gene$SYMBOL
    ),
    by = "gene"
  ) %>%
  # Add t-test results
  left_join(t_test_results, by = "gene") %>%
  # Calculate MA plot values and other metrics
  mutate(
    M = log2(mean_sars/mean_mock),
    A = log2((mean_sars + mean_mock)/2),
    log2FC = M,
    mean_expr = (mean_mock + mean_sars)/2,
    padj = p.adjust(pvalue, method = "BH")
  )

In [ ]:
# Sort by adjusted p-value
sorted_data <- test_data %>%
  arrange(padj, desc(abs(log2FC))) %>%  # First by padj, then by absolute log2FC magnitude
  mutate(rank = row_number())  # Optionally add rank column

# Look at top genes
head(sorted_data %>% 
     select(rank, symbol, mean_expr, log2FC, pvalue, padj),20)

In [ ]:
# Set plot size
options(repr.plot.width=15, repr.plot.height=7)

# Calculate means using abundance this time
mock_abund <- txi$abundance[, grep("Series5_A549_Mock", colnames(txi$abundance))]
sars_abund <- txi$abundance[, grep("Series5_A549_SARS-CoV-2", colnames(txi$abundance))]
mean_mock <- rowMeans(mock_abund)
mean_sars <- rowMeans(sars_abund)

# Create dataframe for plotting
plot_data <- data.frame(
    gene = rownames(txi$abundance),
    mean_mock = mean_mock,
    mean_sars = mean_sars,
    M = log2(mean_sars/mean_mock),
    A = log2((mean_sars + mean_mock)/2)
)

# Create two plots side by side
p1 <- ggplot(plot_data, aes(x = mean_mock + 1, y = mean_sars + 1)) +
    geom_point(alpha = 0.2) +
    geom_abline(color = "red", linetype = "dashed") +
    scale_x_log10() +
    scale_y_log10() +
    theme_minimal() +
    labs(title = "Abundance Comparison (Diagonal Plot)",
         x = "Mock TPM",
         y = "SARS-CoV-2 TPM")

p2 <- ggplot(plot_data, aes(x = A, y = M)) +
    geom_point(alpha = 0.2) +
    geom_hline(yintercept = 0, color = "red", linetype = "dashed") +
    theme_minimal() +
    labs(title = "MA Plot (using TPM)",
         x = "A (average log2 expression)",
         y = "M (log2 fold change)")

p1 + p2

In [ ]:
# Set plot size
options(repr.plot.width=12, repr.plot.height=10)


# 1. Mean-Variance relationship
p1 <- ggplot(test_data, aes(x = log2(mean_expr), y = log2(var_mock))) +
   geom_point(alpha = 0.2) +
   geom_smooth(method = "loess", color = "red") +
   theme_minimal() +
   labs(title = "Mean-Variance Relationship",
        subtitle = "Mock samples - notice increasing variance with mean",
        x = "log2(Mean Expression)",
        y = "log2(Variance)")

# 2. Compare variance between conditions
p2 <- ggplot(test_data, aes(x = log2(var_mock), y = log2(var_sars))) +
   geom_point(alpha = 0.2) +
   geom_abline(intercept = 0, slope = 1, color = "red", linetype = "dashed") +
   theme_minimal() +
   labs(title = "Variance Comparison Between Conditions",
        x = "log2(Mock Variance)",
        y = "log2(SARS Variance)")

# 3. T-statistics vs Expression Level
p3 <- ggplot(test_data, aes(x = log2(mean_expr), y = abs(t_stat))) +
   geom_point(alpha = 0.2) +
   theme_minimal() +
   labs(title = "T-statistics vs Expression Level",
        x = "log2(Mean Expression)",
        y = "Absolute T-statistic")

# 4. Volcano plot
p4 <- ggplot(test_data, aes(x = log2FC, y = -log10(pvalue))) +
   geom_point(alpha = 0.2, aes(color = padj < 0.05)) +
   theme_minimal() +
   labs(title = "Volcano Plot from Simple T-tests",
        x = "log2(Fold Change)",
        y = "-log10(p-value)",
        color = "Significant")

# Using patchwork to display all plots
(p1 + p2) / (p3 + p4)

In [ ]:
# Get gene lengths from txi
gene_lengths <- txi$length[,1]  # take first sample's lengths

# Calculate RPKM
# First get counts
mock_counts <- txi$counts[, grep("Series5_A549_Mock", colnames(txi$counts))]
sars_counts <- txi$counts[, grep("Series5_A549_SARS-CoV-2", colnames(txi$counts))]

# Calculate library sizes
lib_size_mock <- colSums(mock_counts)/1e6  # in millions
lib_size_sars <- colSums(sars_counts)/1e6

# Calculate RPKM
rpkm_mock <- t(t(mock_counts) / lib_size_mock) / (gene_lengths/1000)
rpkm_sars <- t(t(sars_counts) / lib_size_sars) / (gene_lengths/1000)

# Calculate means for both metrics
mean_mock_counts <- rowMeans(mock_counts)
mean_sars_counts <- rowMeans(sars_counts)
mean_mock_rpkm <- rowMeans(rpkm_mock)
mean_sars_rpkm <- rowMeans(rpkm_sars)

# Create plot data
plot_data_counts <- data.frame(
    M = log2((mean_sars_counts + 0.1)/(mean_mock_counts + 0.1)),
    A = log2((mean_sars_counts + mean_mock_counts + 2)/2)
)

plot_data_rpkm <- data.frame(
    M = log2((mean_sars_rpkm + 0)/(mean_mock_rpkm + 0)),
    A = log2((mean_sars_rpkm + mean_mock_rpkm + 0)/2)
)

# Create plots
p1 <- ggplot(plot_data_counts, aes(x = A, y = M)) +
    geom_point(alpha = 0.2) +
    geom_hline(yintercept = 0, color = "red", linetype = "dashed") +
    theme_minimal() +
    labs(title = "MA Plot (Raw Counts)",
         x = "A (average log2 expression)",
         y = "M (log2 fold change)")

p2 <- ggplot(plot_data_rpkm, aes(x = A, y = M)) +
    geom_point(alpha = 0.2) +
    geom_hline(yintercept = 0, color = "red", linetype = "dashed") +
    theme_minimal() +
    labs(title = "MA Plot (RPKM)",
         x = "A (average log2 expression)",
         y = "M (log2 fold change)")

# Display side by side
p1 + p2

In [ ]:
library(tidyverse)

# Convert matrices to long format for ggplot
counts_df <- as.data.frame(txi$counts) %>%
  rownames_to_column("gene_id") %>%
  pivot_longer(-gene_id, 
               names_to = "sample", 
               values_to = "counts")

tpm_df <- as.data.frame(txi$abundance) %>%
  rownames_to_column("gene_id") %>%
  pivot_longer(-gene_id, 
               names_to = "sample", 
               values_to = "tpm")

# Create boxplots for both metrics
p1 <- ggplot(counts_df, aes(x=sample, y=counts + 1)) + 
  geom_boxplot() +
  scale_y_log10() +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Raw Counts Distribution",
       y = "Counts (log10 scale)",
       x = "Sample")

p2 <- ggplot(tpm_df, aes(x=sample, y=tpm + 1)) + 
  geom_boxplot() +
  scale_y_log10() +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "TPM Distribution",
       y = "TPM (log10 scale)",
       x = "Sample")

# Display plots side by side
library(patchwork)
p1 / p2

In [ ]:
# Load required libraries
library(tximport)
library(DESeq2)
library(tidyverse)  # Optional but helpful for data manipulation



# Setup file paths for the new structure
files <- file.path("salmon_quant", sample_info$sample_id, "quant.sf")
names(files) <- sample_info$sample_id

# Verify files exist
all(file.exists(files))

tx2gene_protein_coding <- tx2gene[tx2gene$GENE_TYPE == "protein_coding",]


# Import with tximport
txi <- tximport(files, 
                type = "salmon", 
                tx2gene = tx2gene_protein_coding[,c("TXNAME", "GENEID")],
                ignoreTxVersion = FALSE,
                txOut = FALSE)

# Create DESeq2 object with interaction term
dds <- DESeqDataSetFromTximport(txi,
                               colData = sample_info,
                               design = ~ cell_type + condition + cell_type:condition)

# Set factor levels
dds$condition <- relevel(dds$condition, ref = "Mock")
dds$cell_type <- relevel(dds$cell_type, ref = "A549")

# Run DESeq2
dds <- DESeq(dds)

# Get various results
# 1. Effect of virus in regular A549 cells
res_a549 <- results(dds, 
                   contrast = c("condition", "SARS-CoV-2", "Mock"),
                   alpha = 0.05)

# 2. Effect of virus in ACE2-expressing cells
res_ace2 <- results(dds, 
                   contrast = list(
                       c("condition_SARS.CoV.2_vs_Mock", 
                         "cell_typeA549.ACE2.conditionSARS.CoV.2")))

# 3. Interaction effect (difference in virus effect between cell types)
res_interaction <- results(dds, 
                         name="cell_typeA549.ACE2.conditionSARS.CoV.2")

# Order results by adjusted p-value
res_a549 <- res_a549[order(res_a549$padj), ]
res_ace2 <- res_ace2[order(res_ace2$padj), ]
res_interaction <- res_interaction[order(res_interaction$padj), ]

# Print summaries
print("Virus effect in A549 cells:")
summary(res_a549)
print("\nVirus effect in A549-ACE2 cells:")
summary(res_ace2)
print("\nInteraction effect:")
summary(res_interaction)

In [ ]:
# Look at the patterns in the extra transcripts
salmon_quant <- read_tsv("salmon_quant/SRR11517677_1/quant.sf")
missing_tx <- setdiff(salmon_quant$Name, tx2gene$TXNAME)

# Check some characteristics of missing transcripts
print("Examples of transcripts not in tx2gene:")
head(missing_tx)
print(paste("Number of missing transcripts:", length(missing_tx)))

# Which GENCODE file was used to build the Salmon index?
# (This would help confirm if comprehensive vs basic is the issue)

In [ ]:
library(tximport)
library(DESeq2)

# Read sample information
samples <- read.csv("sample_info.csv")
rownames(samples) <- samples$sample_id

# Get Salmon quant file paths - adjusting for your directory structure
files <- file.path("salmon_quant", paste0(samples$sample_id, "_1"), "quant.sf")
names(files) <- samples$sample_id

# Check if all files exist
all(file.exists(files))

# Import transcript-level counts with tximport
# Using just the TXNAME and GENEID columns from tx2gene
# When using tximport, add txOut=FALSE (which is actually the default)
# Filter tx2gene for protein-coding genes only
tx2gene_protein_coding <- tx2gene[tx2gene$GENE_TYPE == "protein_coding",]

# Use filtered mapping in tximport
txi <- tximport(files, 
                type = "salmon", 
                tx2gene = tx2gene_protein_coding[,c("TXNAME", "GENEID")],
                ignoreTxVersion = FALSE,
                txOut = FALSE)



# Create DESeq2 object
dds <- DESeqDataSetFromTximport(txi,
                               colData = samples,
                               design = ~ condition)

# Set factor levels (make Mock the reference)
dds$condition <- relevel(dds$condition, ref = "Mock")

# Run DESeq2
dds <- DESeq(dds)

# Get results
res <- results(dds, alpha = 0.05)
res <- res[order(res$padj), ]

# Basic summary
summary(res)

In [ ]:
# Check unique gene IDs
n_genes <- length(unique(tx2gene$GENEID))
print(paste("Total unique genes:", n_genes))

# Look at distribution of gene types
gene_type_counts <- table(tx2gene$GENE_TYPE[!duplicated(tx2gene$GENEID)])
print("Gene type distribution:")
print(gene_type_counts)

# Look specifically at protein coding genes
protein_coding_genes <- length(unique(tx2gene$GENEID[tx2gene$GENE_TYPE == "protein_coding"]))
print(paste("Unique protein-coding genes:", protein_coding_genes))

# Check what types of genes are in our DESeq results
res_df <- as.data.frame(res)
res_df$GENEID <- rownames(res_df)

# Merge with gene info
gene_info <- unique(tx2gene[,c("GENEID", "GENE_TYPE")])
res_with_type <- merge(res_df, gene_info, by="GENEID")
res_type_counts <- table(res_with_type$GENE_TYPE)
print("Gene types in DESeq results:")
print(res_type_counts)

In [ ]:
dds$condition

In [ ]:
# Convert DESeq2 results to data frame
res_df <- as.data.frame(res)
res_df$GENEID <- rownames(res_df)

length(res_df[,1])

# Get unique gene info mapping from tx2gene
gene_info <- unique(tx2gene[, c("GENEID", "SYMBOL", "GENE_TYPE")])

# Merge results with gene info
res_annotated <- merge(res_df, gene_info, by="GENEID", all.x=TRUE)

# Order by adjusted p-value
res_annotated <- res_annotated[order(res_annotated$padj), ]

# Take a look at the first few rows
head(res_annotated)

# Basic stats of significant genes
alpha <- 0.05
sig_genes <- sum(!is.na(res_annotated$padj) & res_annotated$padj < alpha)
cat("Significant genes (padj <", alpha, "):", sig_genes, "\n")
head(res)

In [ ]:
library(tidyverse)
res_annotated %>% filter(SYMBOL=="MX1")

In [ ]:
res_annotated %>% filter(padj<0.001, log2FoldChange>0) %>% select(GENEID)


In [ ]:
library(DESeq2)
library(ggplot2)

# MA Plot
plotMA(res, ylim=c(-8,8))

In [ ]:
library(EnhancedVolcano)

# Convert results to dataframe and add gene info
res_df <- as.data.frame(res)
res_df$GENEID <- rownames(res_df)

# Add gene symbols from tx2gene
gene_info <- unique(tx2gene[, c("GENEID", "SYMBOL")])
res_df <- merge(res_df, gene_info, by="GENEID", all.x=TRUE)

# Create volcano plot
EnhancedVolcano(res_df,
    lab = res_df$SYMBOL,
    x = 'log2FoldChange',
    y = 'padj',
    title = 'SARS-CoV-2 vs Mock',
    pCutoff = 0.05,
    FCcutoff = 1,
    pointSize = 1.0,
    labSize = 3.0,
    legendPosition = 'right',
    drawConnectors = TRUE,
    widthConnectors = 0.25)

In [ ]:
total_genes <- nrow(res_df)
total_genes

protein_coding <- length(which(tx2gene$GENE_TYPE == "protein_coding"))
protein_coding

# Count unique protein-coding genes
unique_protein_coding <- length(unique(tx2gene$GENEID[tx2gene$GENE_TYPE == "protein_coding"]))
print(paste("Unique protein coding genes:", unique_protein_coding))

# See the distribution of transcripts per gene
transcripts_per_gene <- table(tx2gene$GENEID[tx2gene$GENE_TYPE == "protein_coding"])
summary(as.numeric(transcripts_per_gene))

In [ ]:
# DESeq2 analysis of NHBE SARS-CoV-2 RNA-seq data
library(tximport)
library(DESeq2)
library(tidyverse)
library(pheatmap)
library(EnhancedVolcano)

# Read sample information
samples <- read.table("salmon_quant/sample_info.txt", header=TRUE)
rownames(samples) <- samples$sample

# Get Salmon quant file paths
files <- file.path("salmon_quant", samples$sample, "quant.sf")
names(files) <- samples$sample

# Import transcript-level counts with tximport
tx2gene <- read.csv("reference/tx2gene.csv")  # You'll need to create this mapping file
txi <- tximport(files, type="salmon", tx2gene=tx2gene, ignoreTxVersion=TRUE)

# Create DESeq2 object
dds <- DESeqDataSetFromTximport(txi,
                               colData = samples,
                               design = ~ condition)

# Set factor levels (make Mock the reference)
dds$condition <- relevel(dds$condition, ref = "Mock")

# Run DESeq2
dds <- DESeq(dds)

# Get results
res <- results(dds, alpha = 0.05)
res <- res[order(res$padj), ]

# Basic statistics
summary(res)

# Export results
write.csv(res, "deseq2_results.csv")

# MA plot
png("plots/MA_plot.png", width=800, height=600)
plotMA(res, ylim=c(-5,5))
dev.off()

# Volcano plot
png("plots/volcano_plot.png", width=1000, height=800)
EnhancedVolcano(res,
                lab = rownames(res),
                x = 'log2FoldChange',
                y = 'padj',
                pCutoff = 0.05,
                FCcutoff = 1,
                title = 'NHBE SARS-CoV-2 vs Mock')
dev.off()

# PCA plot
vsd <- vst(dds, blind=FALSE)
png("plots/PCA_plot.png", width=800, height=600)
plotPCA(vsd, intgroup="condition") +
  theme_minimal() +
  ggtitle("PCA of NHBE samples")
dev.off()

# Heatmap of top genes
select <- order(rowMeans(counts(dds,normalized=TRUE)),
               decreasing=TRUE)[1:50]
mat <- assay(vsd)[select,]
png("plots/heatmap_top50.png", width=1000, height=1200)
pheatmap(mat,
         annotation_col=data.frame(Condition=samples$condition),
         show_rownames=TRUE,
         labels_row=rownames(mat),
         scale="row",
         cluster_rows=TRUE,
         cluster_cols=TRUE,
         main="Top 50 genes by expression")
dev.off()

# Get DE genes with specific thresholds
de_genes <- subset(res, padj < 0.05 & abs(log2FoldChange) > 1)
write.csv(de_genes, "significant_DE_genes.csv")

# Print summary of findings
cat("Number of DE genes (padj < 0.05 & |log2FC| > 1):", nrow(de_genes), "\n")
cat("Up-regulated:", sum(de_genes$log2FoldChange > 0), "\n")
cat("Down-regulated:", sum(de_genes$log2FoldChange < 0), "\n")

In [ ]:
options(jupyter.plot_mimetypes = 'image/png')
options(repr.plot.width = 8, repr.plot.height = 8, repr.plot.res=200)

# Prepare annotation data
annotation_col <- data.frame(
    Condition = samples$condition,
    row.names = samples$sample
)

# Define colors explicitly
ann_colors <- list(
    Condition = c(Mock = "#1B9E77", "SARS-CoV-2" = "#D95F02")
)

# Create heatmap with explicit color settings
pheatmap(mat,
         annotation_col = annotation_col,
         annotation_colors = ann_colors,
         show_rownames = TRUE,
         show_colnames = TRUE,
         scale = "row",
         cluster_rows = TRUE,
         cluster_cols = TRUE,
         main = "Top 50 genes by expression")

In [ ]:
library(DESeq2)
library(ggplot2)

# MA Plot
plotMA(res_a549, ylim=c(-8,8))
plotMA(res_ace2, ylim=c(-8,8))
plotMA(res_interaction, ylim=c(-8,8))

In [ ]:


library(clusterProfiler)
library(org.Hs.eg.db)
library(enrichplot)
library(tidyverse)
library(gprofiler2)

# Let's prepare our results for enrichment analysis
# First for the interaction results
interaction_results <- as.data.frame(res_interaction) %>%
  rownames_to_column("ENSEMBL") %>%
  # Filter for significant genes
  filter(!is.na(padj), padj < 0.05) %>%
  # Remove version numbers if needed
  mutate(ENSEMBL = str_replace(ENSEMBL, "\\.[0-9]*$", ""))

# Get significant genes ordered by fold change
interaction_genes <- interaction_results %>%
  arrange(desc(log2FoldChange)) %>%
  pull(ENSEMBL)

# Run GO enrichment
ego <- enrichGO(gene = interaction_genes,
                OrgDb = org.Hs.eg.db,
                keyType = "ENSEMBL",
                ont = "BP",  # Biological Process
                pAdjustMethod = "BH",
                pvalueCutoff = 0.05)

# Create a nice visualization
# Dotplot of top enriched terms
dotplot_go <- ego %>%
  as_tibble() %>%
  slice_head(n = 20) %>%  # Top 20 terms
  ggplot(aes(x = Count, y = reorder(Description, Count))) +
  geom_point(aes(size = Count, color = p.adjust)) +
  scale_color_gradient(low = "red", high = "blue") +
  theme_minimal() +
  labs(x = "Gene Count",
       y = "GO Term",
       title = "Top GO Terms Enriched in Interaction Effect",
       color = "Adjusted\np-value")

# For GSEA, we'll need to create a ranked list
gsea_ranking <- res_interaction %>%
  as.data.frame() %>%
  rownames_to_column("ENSEMBL") %>%
  filter(!is.na(stat)) %>%
  mutate(ENSEMBL = str_replace(ENSEMBL, "\\.[0-9]*$", "")) %>%
  arrange(desc(stat)) %>%
  pull(stat, name = ENSEMBL)

# Run GSEA
gse <- gseGO(geneList = gsea_ranking,
             OrgDb = org.Hs.eg.db,
             keyType = "ENSEMBL",
             ont = "BP",
             minGSSize = 10,
             maxGSSize = 500,
             pvalueCutoff = 0.05,
             verbose = FALSE)

# Visualize GSEA results
gsea_plot <- gseaplot2(gse, geneSetID = 1:3, pvalue_table = TRUE)

# Create a tidy results table
go_results <- ego %>%
  as_tibble() %>%
  arrange(p.adjust) %>%
  dplyr::select(Description, GeneRatio, pvalue, p.adjust, Count, geneID)

gsea_results <- gse %>%
  as_tibble() %>%
  arrange(pvalue) %>%
  dplyr::select(Description, NES, pvalue, p.adjust, core_enrichment)

# Save results
write_csv(go_results, "interaction_GO_results.csv")
write_csv(gsea_results, "interaction_GSEA_results.csv")

# Print top results
print("Top GO Terms:")
print(head(go_results %>% dplyr::select(Description, p.adjust, Count), 10))

print("\nTop GSEA Results:")
print(head(gsea_results %>% dplyr::select(Description, NES, p.adjust), 10))

In [ ]:
print(gsea_plot)
print(dotplot_go)

In [ ]:
# Let's look at raw counts first
# Extract just A549 Mock and SARS-CoV-2 samples
a549_samples <- sample_info %>%
  filter(cell_type == "A549") %>%
  pull(sample_id)

# Get raw counts for these samples
raw_counts <- txi$counts[, a549_samples]

# Let's look at some basic stats
# 1. Simple ratios (this will show why we need normalization)
mock_means <- rowMeans(raw_counts[, 1:3])
sars_means <- rowMeans(raw_counts[, 4:6])
simple_ratios <- sars_means / mock_means

# Visualize total counts per sample (sequencing depth differences)
colSums_plot <- data.frame(
  sample = colnames(raw_counts),
  total_counts = colSums(raw_counts)
) %>%
  ggplot(aes(x = sample, y = total_counts)) +
  geom_bar(stat = "identity") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Raw Library Sizes",
       y = "Total Counts",
       x = "Sample")

# Look at gene lengths too (why we need RPKM/TPM)
# We'll need gene lengths from your GTF file
# For now let's just look at count distributions
count_distributions <- data.frame(
  counts = as.vector(raw_counts),
  sample = rep(colnames(raw_counts), each = nrow(raw_counts))
) %>%
  ggplot(aes(x = log2(counts + 1), fill = sample)) +
  geom_density(alpha = 0.5) +
  theme_minimal() +
  labs(title = "Raw Count Distributions",
       x = "log2(counts + 1)",
       y = "Density")

# Show these plots
print(colSums_plot)
print(count_distributions)

# Let's look at some problematic cases
problems <- data.frame(
  mock_mean = mock_means,
  sars_mean = sars_means,
  ratio = simple_ratios
) %>%
  rownames_to_column("gene") %>%
  mutate(
    avg_expression = (mock_mean + sars_mean)/2,
    log2_ratio = log2(ratio)
  ) %>%
  filter(!is.infinite(log2_ratio), !is.na(log2_ratio))

# Basic MA plot with raw values
raw_ma <- ggplot(problems, aes(x = log2(avg_expression), y = log2_ratio)) +
  geom_point(alpha = 0.2) +
  theme_minimal() +
  labs(title = "Raw MA Plot (before any normalization)",
       x = "log2 Average Expression",
       y = "log2 Ratio (SARS/Mock)")

print(raw_ma)